In [125]:
import json 
import csv
import pandas as pd
import datetime, time

In [372]:
brand = pd.read_json('/Users/katie/Desktop/Resume/Fetch Rewards/brands.json',lines=True)
receipts = pd.read_json('/Users/katie/Desktop/Resume/Fetch Rewards/receipts.json',lines=True)
users = pd.read_json('/Users/katie/Desktop/Resume/Fetch Rewards/users.json',lines=True)

# Brand table

In [308]:
brand

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827
...,...,...,...,...,...,...,...,...
1162,{'$oid': '5f77274dbe37ce6b592e90c0'},511111116752,Baking,BAKING,"{'$ref': 'Cogs', '$id': {'$oid': '5f77274dbe37...",test brand @1601644365844,NaN,NaN
1163,{'$oid': '5dc1fca91dda2c0ad7da64ae'},511111706328,Breakfast & Cereal,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '53e10d6368ab...",Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL
1164,{'$oid': '5f494c6e04db711dd8fe87e7'},511111416173,Candy & Sweets,CANDY_AND_SWEETS,"{'$ref': 'Cogs', '$id': {'$oid': '5332fa12e4b0...",test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217
1165,{'$oid': '5a021611e4b00efe02b02a57'},511111400608,Grocery,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5f6e4b0...",LIPTON TEA Leaves,0.0,LIPTON TEA Leaves


First, the id column is in json format, so I am going to modity it.

In [309]:
brand['_id'] = brand['_id'].apply(lambda x: x['$oid'])

According to the brand table, I find that there are mutiple elements in cpg column, so I will extract them into a different column. In this section, I use json normalize function in pandas.

In [310]:
brand_cpg = pd.json_normalize(brand['cpg'])

We can see that cpg column has been spilted into $ref and id.oid

In [311]:
brand_cpg

,$ref,$id.$oid
0,Cogs,601ac114be37ce2ead437550
1,Cogs,5332f5fbe4b03c9a25efd0ba
2,Cogs,601ac142be37ce2ead437559
3,Cogs,601ac142be37ce2ead437559
4,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...
1162,Cogs,5f77274dbe37ce6b592e90bf
1163,Cogs,53e10d6368abd3c7065097cc
1164,Cogs,5332fa12e4b03c9a25efd1e7
1165,Cogs,5332f5f6e4b03c9a25efd0b4


Now I am going to merge the brand and brand_cpg together.

In [314]:
brand_merge = pd.merge(brand, brand_cpg,left_on=None, right_on=None, left_index=True, right_index=True,how='outer')

In [450]:
brand_merge

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,$ref,$id.$oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...,...,...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,Baking,BAKING,"{'$ref': 'Cogs', '$id': {'$oid': '5f77274dbe37...",test brand @1601644365844,NaN,NaN,Cogs,5f77274dbe37ce6b592e90bf
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '53e10d6368ab...",Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,Cogs,53e10d6368abd3c7065097cc
1164,5f494c6e04db711dd8fe87e7,511111416173,Candy & Sweets,CANDY_AND_SWEETS,"{'$ref': 'Cogs', '$id': {'$oid': '5332fa12e4b0...",test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,Cogs,5332fa12e4b03c9a25efd1e7
1165,5a021611e4b00efe02b02a57,511111400608,Grocery,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5f6e4b0...",LIPTON TEA Leaves,0.0,LIPTON TEA Leaves,Cogs,5332f5f6e4b03c9a25efd0b4


# Receipts Table

In [373]:
receipts

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,{'$oid': '603cc0630a720fde100003e6'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614594147000},{'$date': 1614594147000},NaN,{'$date': 1614594148000},NaN,25.0,{'$date': 1597622400000},2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,{'$oid': '603d0b710a720fde1000042a'},NaN,NaN,{'$date': 1614613361873},{'$date': 1614613361873},NaN,{'$date': 1614613361873},NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,{'$oid': '603cf5290a720fde10000413'},NaN,NaN,{'$date': 1614607657664},{'$date': 1614607657664},NaN,{'$date': 1614607657664},NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,{'$date': 1597622400000},2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


Same with brand table, I first change the id column into _id.

In [380]:
receipts['_id'] = receipts['_id'].apply(lambda x: x['$oid'])

Since the data is not formated in the dataframe, so I use get_time_from_timestamp function to adjust date format.

In [382]:
def get_time_from_timestamp(x):
    try:
        return(datetime.fromtimestamp(int(x['$date'])//1000).strftime('%Y/%m/%d %H:%M:%S'))
    except TypeError:
        return(None)

In [383]:
receipts['createDate'] = receipts['createDate'].apply(lambda x: get_time_from_timestamp(x))
receipts['dateScanned'] = receipts['dateScanned'].apply(lambda x: get_time_from_timestamp(x))
receipts['modifyDate'] = receipts['modifyDate'].apply(lambda x: get_time_from_timestamp(x))
receipts['finishedDate'] = receipts['finishedDate'].apply(lambda x: get_time_from_timestamp(x))
receipts['pointsAwardedDate'] = receipts['pointsAwardedDate'].apply(lambda x: get_time_from_timestamp(x))
receipts['purchaseDate'] = receipts['purchaseDate'].apply(lambda x: get_time_from_timestamp(x))

Then, the rewardsReceiptItemList column contains mutipule elements, I will need to extract them into a new column.

In [459]:

receipts = receipts.explode('rewardsReceiptItemList')
receipts = receipts.fillna({'rewardsReceiptItemList':'{}'})
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(lambda x:str(x))
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(literal_eval)
receipts_norm = pd.json_normalize(receipts['rewardsReceiptItemList'],errors='ignore',record_prefix='rewardsReceiptItemList')\
.add_prefix('rewardsReceiptItemList.')

KeyError: 0

First, I fill the NaN value with blank { }. Then, I change the type to string. 

In [460]:
receipts = receipts.fillna({'rewardsReceiptItemList':'{}'})
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(lambda x:str(x))

Because, I get an error if I normalize receipts without I use literal_eval because I need eval to evaluate input expression.

In [461]:
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(literal_eval)

In [467]:
receipts_rewardsReceiptItemList = pd.json_normalize(receipts['rewardsReceiptItemList'],errors='ignore',record_prefix='rewardsReceiptItemList')\
.add_prefix('rewardsReceiptItemList.')

In [464]:
receipts

,level_0,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,0,0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021/01/03 10:25:31,2021/01/03 10:25:31,2021/01/03 10:25:31,2021/01/03 10:25:36,2021/01/03 10:25:31,500.0,2021/01/02 19:00:00,5.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,1,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021/01/03 10:24:43,2021/01/03 10:24:43,2021/01/03 10:24:43,2021/01/03 10:24:48,2021/01/03 10:24:43,150.0,2021/01/02 10:24:43,2.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,2,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021/01/03 10:24:43,2021/01/03 10:24:43,2021/01/03 10:24:43,2021/01/03 10:24:48,2021/01/03 10:24:43,150.0,2021/01/02 10:24:43,2.0,"{'barcode': '028400642255', 'description': 'DO...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
3,3,2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021/01/03 10:25:37,2021/01/03 10:25:37,None,2021/01/03 10:25:42,None,5.0,2021/01/02 19:00:00,1.0,"{'needsFetchReview': False, 'partnerItemId': '...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
4,4,3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021/01/03 10:25:34,2021/01/03 10:25:34,2021/01/03 10:25:34,2021/01/03 10:25:39,2021/01/03 10:25:34,5.0,2021/01/02 19:00:00,4.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,7376,1115,603d0b710a720fde1000042a,NaN,NaN,2021/03/01 10:42:41,2021/03/01 10:42:41,None,2021/03/01 10:42:41,None,NaN,None,NaN,{},SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
7377,7377,1116,603cf5290a720fde10000413,NaN,NaN,2021/03/01 09:07:37,2021/03/01 09:07:37,None,2021/03/01 09:07:37,None,NaN,None,NaN,{},SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
7378,7378,1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021/03/01 08:07:28,2021/03/01 08:07:28,None,2021/03/01 08:07:29,None,25.0,2020/08/16 20:00:00,2.0,"{'barcode': 'B076FJ92M4', 'description': 'muel...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
7379,7379,1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021/03/01 08:07:28,2021/03/01 08:07:28,None,2021/03/01 08:07:29,None,25.0,2020/08/16 20:00:00,2.0,"{'barcode': 'B07BRRLSVC', 'description': 'thin...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [469]:
receipts_merge = pd.merge(receipts, receipts_rewardsReceiptItemList,left_on=None, right_on=None, left_index=True, right_index=True,how='outer')

Because I will execute SQL query later, so I modify . to _ to avoid failure on SQL.

In [470]:
receipts_merge.columns = receipts_merge.columns.str.replace(".", "_")

In [471]:
receipts_merge

,level_0,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,...,rewardsReceiptItemList_itemNumber,rewardsReceiptItemList_originalMetaBriteQuantityPurchased,rewardsReceiptItemList_pointsEarned,rewardsReceiptItemList_targetPrice,rewardsReceiptItemList_competitiveProduct,rewardsReceiptItemList_originalFinalPrice,rewardsReceiptItemList_originalMetaBriteItemPrice,rewardsReceiptItemList_deleted,rewardsReceiptItemList_priceAfterCoupon,rewardsReceiptItemList_metabriteCampaignId
0,0,0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021/01/03 10:25:31,2021/01/03 10:25:31,2021/01/03 10:25:31,2021/01/03 10:25:36,2021/01/03 10:25:31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021/01/03 10:24:43,2021/01/03 10:24:43,2021/01/03 10:24:43,2021/01/03 10:24:48,2021/01/03 10:24:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021/01/03 10:24:43,2021/01/03 10:24:43,2021/01/03 10:24:43,2021/01/03 10:24:48,2021/01/03 10:24:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021/01/03 10:25:37,2021/01/03 10:25:37,None,2021/01/03 10:25:42,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021/01/03 10:25:34,2021/01/03 10:25:34,2021/01/03 10:25:34,2021/01/03 10:25:39,2021/01/03 10:25:34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,7376,1115,603d0b710a720fde1000042a,NaN,NaN,2021/03/01 10:42:41,2021/03/01 10:42:41,None,2021/03/01 10:42:41,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7377,7377,1116,603cf5290a720fde10000413,NaN,NaN,2021/03/01 09:07:37,2021/03/01 09:07:37,None,2021/03/01 09:07:37,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7378,7378,1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021/03/01 08:07:28,2021/03/01 08:07:28,None,2021/03/01 08:07:29,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
7379,7379,1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021/03/01 08:07:28,2021/03/01 08:07:28,None,2021/03/01 08:07:29,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN


# Users Table

In [431]:
users

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
...,...,...,...,...,...,...,...
490,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
491,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
492,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
493,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN


Modify column $oid to _id

In [432]:
users['_id'] = users['_id'].apply(lambda x: x['$oid'])

Then, convert the time to the format datetime

In [433]:
users['createdDate'] = users['createdDate'].apply(lambda x: get_time_from_timestamp(x))
users['lastLogin'] = users['lastLogin'].apply(lambda x: get_time_from_timestamp(x))

In [434]:
users

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021/01/03 10:24:04,2021/01/03 10:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021/01/03 10:24:04,2021/01/03 10:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021/01/03 10:24:04,2021/01/03 10:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021/01/03 10:25:30,2021/01/03 10:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021/01/03 10:24:04,2021/01/03 10:25:37,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,2014/12/19 09:21:22,2021/03/05 11:52:23,fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,2014/12/19 09:21:22,2021/03/05 11:52:23,fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,2014/12/19 09:21:22,2021/03/05 11:52:23,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,2014/12/19 09:21:22,2021/03/05 11:52:23,fetch-staff,NaN,NaN


Finally, I export them into csv file and the for the next step, I will execute sql query using PySpark on Databrick.

In [451]:
brand_merge.to_csv('brands.csv')

In [472]:
receipts_merge.to_csv('receipts.csv')

In [454]:
users.to_csv('users.csv')